### Group 8
- Nimrod Grandpierre
- Jonas Riber Jørgensen
- Johan Ulstrup
- Laura Fur

# Project 5: NJ tree construction
This project is about making an efficient implementation of the neighbor-joining (NJ) algorithm as shown on slide 50 in the slides about tree reconstruction and compare its performance to the NJ programs QuickTree and RapidNJ that you know from project 4.

## <span style="color:cornflowerblue">Problem<span/>
You should make a program that implements the NJ algorithm as shown in the slides about tree reconstruction. Your program program should take a distance matrix in phylip-format as input and produce a tree in newick-format as output. You should know these formats from project 4. Your aim is to make your implementation as efficient as possible.

The file example_slide4.phy contains the distance matrix (in phylip-format) from slide 4 in the slides about tree reconstruction. With this matrix as input, your program should produce the tree that is also shown on slide 4 in the slides about tree reconstruction.

In [4]:
import pandas as pd
import numpy as np
from read_phylip import *
from correct_DM import *
from NJ_print import *
import os

### Implementation

In [5]:
def NJ(S: list, DM: dict):

    T = []

    while len(S) > 3:

        def cluster(S, DM):
            nonlocal T

            """ Step 1: a) Correcting DM b) Choosing pair to cluster """
            N = compute_N(S, DM)

            min_distances = {i: min((distance, idx) for idx, distance in enumerate(row) if distance != 0.0) for i, row in N.items()}  # Finding the shortest distance and its index for each sequence in N.
            S1 = min(min_distances, key = lambda k: min_distances[k][0])                                                              # Key with the minimum value among all minimum distances.
            S1_idx = next((i for i, seq in enumerate(DM) if seq == S1))
            S2_idx = min_distances[S1][1]                                                                                             # Extract the index of the second sequence. 
            S2 = S[S2_idx]                                                                                                            # Get name for the second sequence from S.

            k = [S1, S2]

            """ Step 2: Adding new node, k, to T """
            k_copy = k.copy()  # Create a copy of k to avoid overwriting.
            T.append(k_copy)   # Append the copy to T

            """ Step 3: Calculating distances to new cluster and adding edges, (k, i) and (k, j) """
            
            # Adding edges:
            dij = {seq_i: DM[seq_i] for seq_i in DM.keys()}
            ri = {seq_i: round((1 / (len(DM) - 2)) * np.sum(DM_i), 2) for i, (seq_i, DM_i) in enumerate(DM.items()) for j in range(len(DM_i)) if i != j}
            
            diu = round(0.5 * dij[S1][S2_idx] + 0.5 * (ri[S1] - ri[S2]), 2)      # Distance from S1 to common node in cluster.
            dju = round(dij[S1][S2_idx] - diu, 2)                                # Distance from S2 to common node in cluster.

            cluster_idx = T.index(k)
            T[cluster_idx].append(diu)
            T[cluster_idx].append(dju)

            # Calculating distances from other sequences to the new cluster:
            dijk = {}
            for i, seq_i in enumerate(DM.keys()):
                if seq_i not in (S1, S2):
                    dik = DM[S1][i]
                    djk = DM[S2][i]
                    dij = DM[S1][S2_idx]
                    dijk[seq_i] = round(0.5 * (dik + djk - dij), 2)
                dijk[(S1, S2)] = 0                                               # Converting cluster list to tuple, as lists are not allowed as keys.

            """ Step 4: Update distance matrix """
            UDM = DM.copy()                                                      # Creating temporary distance matrix.
            UDM[(S1, S2)] = UDM.pop(S1)                                          # Overwrite key for first sequence. Note that this puts the (S1, S2) key in the last position of the dictionary.

            del UDM[S2]                                                          # Delete key for the second sequence.

            for seq, dists in UDM.items():
                UDM[seq] = np.delete(dists, list(DM.keys()).index(S2))           # Delete column for the second sequence.
            
            DM_idxs = {seq_i: i for i, seq_i in enumerate(DM.keys())}            # Sequence indexes in the input distance matrix.

            for i, seq_i in enumerate(UDM.keys()):
                if seq_i != (S1, S2):
                    for j, seq_j in enumerate(UDM.keys()):
                        if seq_j != seq_i:
                            if seq_j == (S1, S2):
                                UDM[seq_i][j] = dijk[seq_i]                      # Distance to k.
                            if seq_j != (S1, S2):
                                UDM[seq_i][j] = DM[seq_i][DM_idxs[seq_j]]        # Distance to other sequence.
                        if seq_j == seq_i:
                            UDM[seq_i][j] = 0                                    # Distance to self - always 0.
                if seq_i == (S1, S2):
                    for j, seq_j in enumerate(UDM.keys()):
                        UDM[seq_i][j] = dijk[seq_j]                              # Distances from k.
                                        
            """ Step 5: Delete i and j from S and add the new taxon, k, to S. """
            
            S.remove(S1)
            S.remove(S2)
            S.append((S1, S2))

            return S, UDM, k

        S, DM, k = cluster(S, DM)  # k is the last cluster formed.

    """ Termination """
    i, j, m = S

    # Ensure m is k, otherwise swap the variable that is k for m.
    if m != tuple(k):
        if i == tuple(k):
            i, m = m, i
        elif j == k:
            j, m = m, j

    idx_i = list(DM.keys()).index(i)
    idx_j = list(DM.keys()).index(j)
    idx_m = list(DM.keys()).index(m)

    gamma_vi = round((DM[i][idx_j] + DM[i][idx_m] - DM[j][idx_m])/2, 3)
    gamma_vj = round((DM[j][idx_i] + DM[j][idx_m] - DM[i][idx_m])/2, 3)
    gamma_vm = round((DM[m][idx_i] + DM[m][idx_j] - DM[i][idx_j])/2, 3)

    T.append([i, gamma_vi])
    T.append([j, gamma_vj])
    T.append([m, gamma_vm])

    return T

## <span style = 'color:cornflowerblue'>Tests<span/>

Simple testdata with four sequences from ```wiki_ex.phy```

In [6]:
wiki_ex = initiate_DM('wiki_ex.phy')
wiki_ex_taxa = [i for i in wiki_ex.keys()]

wiki_ex_out = NJ_print(wiki_ex_taxa, wiki_ex)
wiki_ex_out

Initialization
No. taxa:          4
Taxa:              ['A', 'B', 'C', 'D']
DM:                {'A': array([ 0., 17., 21., 27.]), 'B': array([17.,  0., 12., 18.]), 'C': array([21., 12.,  0., 14.]), 'D': array([27., 18., 14.,  0.])}
Corrected DM:      {'A': array([  0., -39., -35., -35.]), 'B': array([-39.,   0., -35., -35.]), 'C': array([-35., -35.,   0., -39.]), 'D': array([-35., -35., -39.,   0.])}
Pair to cluster:   ['A', 'B'] 

Clustering
Distance to node:  A: 13.0   B: 4.0
Distance to k:     {('A', 'B'): 0, 'C': 8.0, 'D': 14.0}
Overwrite S1:      {'B': array([17.,  0., 12., 18.]), 'C': array([21., 12.,  0., 14.]), 'D': array([27., 18., 14.,  0.]), ('A', 'B'): array([ 0., 17., 21., 27.])}
Delete S2:         {'C': array([21.,  0., 14.]), 'D': array([27., 14.,  0.]), ('A', 'B'): array([ 0., 21., 27.])}
Updated DM:        {'C': array([ 0., 14.,  8.]), 'D': array([14.,  0., 14.]), ('A', 'B'): array([ 8., 14.,  0.])} 



[['A', 'B', 13.0, 4.0], ['C', 4.0], ['D', 10.0], [('A', 'B'), 4.0]]

For this example dataset, the Newick format should look like this:

(C: 4.0, D: 10.0,(A: 13.0, B: 4.0): 4.0);

Provided testdata with five sequences from ```example_slide4.phy```

In [7]:
slide4_ex = initiate_DM('example_slide4.phy')
slide4_ex_taxa = [i for i in slide4_ex.keys()]

slide4_ex_out = NJ_print(slide4_ex_taxa, slide4_ex)
slide4_ex_out

Initialization
No. taxa:          5
Taxa:              ['A', 'B', 'C', 'D', 'E']
DM:                {'A': array([0.  , 0.23, 0.16, 0.2 , 0.17]), 'B': array([0.23, 0.  , 0.23, 0.17, 0.24]), 'C': array([0.16, 0.23, 0.  , 0.2 , 0.11]), 'D': array([0.2 , 0.17, 0.2 , 0.  , 0.21]), 'E': array([0.17, 0.24, 0.11, 0.21, 0.  ])}
Corrected DM:      {'A': array([ 0.  , -0.31, -0.32, -0.31, -0.32]), 'B': array([-0.31,  0.  , -0.29, -0.38, -0.29]), 'C': array([-0.32, -0.29,  0.  , -0.29, -0.36]), 'D': array([-0.31, -0.38, -0.29,  0.  , -0.29]), 'E': array([-0.32, -0.29, -0.36, -0.29,  0.  ])}
Pair to cluster:   ['B', 'D'] 

Clustering
Distance to node:  B: 0.1   D: 0.07
Distance to k:     {'A': 0.13, ('B', 'D'): 0, 'C': 0.13, 'E': 0.14}
Overwrite S1:      {'A': array([0.  , 0.23, 0.16, 0.2 , 0.17]), 'C': array([0.16, 0.23, 0.  , 0.2 , 0.11]), 'D': array([0.2 , 0.17, 0.2 , 0.  , 0.21]), 'E': array([0.17, 0.24, 0.11, 0.21, 0.  ]), ('B', 'D'): array([0.23, 0.  , 0.23, 0.17, 0.24])}
Delete S2:         {

[['B', 'D', 0.1, 0.07],
 ['A', ('B', 'D'), 0.08, 0.05],
 ['C', 0.05],
 ['E', 0.06],
 [('A', ('B', 'D')), 0.03]]

For this example dataset, the Newick format should look like this:

(C: 0.5000, E: 0.6000, (A: 0.8000, (B: 1.0000, D: 0.7000): 0.5000): 0.3000);

## <span style = 'color:cornflowerblue'>Experiments<span/>
From project 4, you know the programs QuickTree and RapidNJ that are implementations of the NJ methods. QuickTree implements the basic cubic time algorithm while RapidNJ implements an algorithm the is faster in practice.

You should compare the performance of your program against these two program in the following way.

The archive unique_distance_matrices.zip contains 14 distance matrices (in phylip-format) ranging in size from 89 to 1849 species. For each distance matrix, you should do the following:

1. Measure the time it takes to construct the corresponding NJ tree using QuickTree, RapidNJ, and your program.
2. Compute the RF-distances (using your program rfdist from project 4) between the trees produced by QuickTree, RapidNJ, and your program.

(If you want to investigate the running time of your program on more examples than provided in distance_matrices.zip, then you are welcome to download pfam_alignments.zip that contains 128 of alignment in Stockholm-format (from the Pfam database) aligning from 58 to 71535 species that you can convert to distance matrices in phylip-format using e.g. QuickTree. However, converting the big alignments to distance matrices would probably take too long and require too much space.)

In [ ]:
uniqueDistMatrices = sorted([file for file in os.listdir('unique_distance_matrices')], key=lambda x: int(x.split('_')[0]))

In [ ]:
data = initiate_DM(f'unique_distance_matrices/{uniqueDistMatrices[1]}')
data_taxa = [i for i in data.keys()]

NJ(data_taxa, data)